In [1]:
import sys
sys.path.insert(0, '../src')
from model import UNetAutoencoder
from dataset import SemiSupervisedMnistDataModule

In [2]:
autoencoder_a = UNetAutoencoder()  # no entrenado

print("  - Encoder: ", autoencoder_a)

  - Encoder:  UNetAutoencoder(
  (enc1): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (enc2): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bottleneck): Sequential(
    (0): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 256, kernel_size=(3, 

In [3]:
dm = SemiSupervisedMnistDataModule(
    data_dir="../data/",
    batch_size=64,
    num_workers=4,
    label_pct=0.3  # usa 0.1 para el segundo experimento
)
dm.setup()

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

# --- Configuraciones básicas ---
MAX_EPOCHS = 20
EARLY_STOP_PATIENCE = 3
PROJECT_NAME = "autoencoder-clasificador-mnist"

# --- Logger con Weights & Biases ---
wandb_logger = WandbLogger(project=PROJECT_NAME, log_model=True)

# --- EarlyStopping ---
early_stop_callback = EarlyStopping(
    monitor="val_loss",  # o "val_accuracy", según tu modelo
    patience=EARLY_STOP_PATIENCE,
    verbose=True,
    mode="min"
)

# --- Checkpoint opcional ---
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    save_top_k=1,
    mode="min",
    dirpath="../output/checkpoints/",
    filename="best-model-{epoch:02d}-{val_loss:.2f}",
)

# --- Trainer ---
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    logger=wandb_logger,
    callbacks=[early_stop_callback, checkpoint_callback],
    accelerator="auto",  # usa GPU si está disponible
    devices="auto",      # auto-detecta CPU/GPU
    log_every_n_steps=10
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [5]:
# Entrenar autoencoder con datos no etiquetados
trainer.fit(autoencoder_a, train_dataloaders=dm.unlabeled_dataloader(), val_dataloaders=dm.val_dataloader())


You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: tdummy482 (tdummy482-tec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


c:\Users\ncque\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\ncque\OneDrive\Documentos\IA\Proyecto_3_IA\output\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type               | Params | Mode 
-----------------------------------------------------------
0  | enc1       | DoubleConv         | 38.7 K | train
1  | pool1      | MaxPool2d          | 0      | train
2  | enc2       | DoubleConv         | 221 K  | train
3  | pool2      | MaxPool2d          | 0      | train
4  | bottleneck | Sequential         | 918 K  | train
5  | up2        | ConvTranspose2d    | 65.7 K | train
6  | dec2       | DoubleConv         | 442 K  | train
7  | up1        | ConvTranspose2d    | 32.8 K | train
8  | dec1       | DoubleConv         | 110 K  | train
9  | final_conv | Conv2d             | 195    | train
10 | loss_fn    | MSELoss            | 0      | train
11 | a

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\ncque\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [64, 49152]